# Part-of-Speech Tagging Practical Session (Solutions)

This part was produced using [Jupyter](http://jupyter.org).  
If you are used to it, you can [download the corresponding notebook code from here](TP3-PoSTagging.ipynb). If not, no problem at all, this is not mandatory at all: simply proceed as usual in your favorite Python environment.

## Introduction

In this practical session, our goals are to:

* better understand the role and mechanisms behind PoS tagging;
* explore applications of PoS tagging such as dealing with ambiguity or vocabulary reduction;
* get accustomed to the Viterbi algorithm through a concrete example.

This practical session is making use of the NLTk. Please refer to [this part of first practical session](TP1.php#nltk) for a setup.

## 1. Parts of Speech and Ambiguity

For this exercise, we will be using the basic functionality of the built-in PoS tagger from NLTK. All you need to know for this part can be found in [section 1  of chapter 5 of the NLTK book](http://www.nltk.org/book/ch05.html#using-a-tagger).

1. Search the web for "ambiguous headlines", to find such gems as: "_British Left Waffles on Falkland Islands_", and "_Juvenile Court to Try Shooting Defendant_". [That website (link)](http://www.ling.upenn.edu/~beatrice/humor/headlines.html) has some good ones.
1. Manually tag (by yourself) these headlines.
   Do you think the information about part-of-speech tags would help a first reading?
1. Use the built-in NLTK PoS tagger on the headlines.
   Does it find the correct tags in these cases?

**Note:** NLTK provides documentation for each tag, which can be queried using the tag, e.g.

    nltk.help.upenn_tagset('RB')

or some regular expression, e.g.

    nltk.help.upenn_tagset('NN.*')

In [ ]:
import nltk

# some headline
headline = "British Left Waffles on Falkland Islands"

[**to do:** complete the above code to tag your headline phrase]

In [ ]:
# Tokenize and then tag the headline
tagged = nltk.pos_tag( nltk.word_tokenize( headline ) );
print( tagged );
 
# some help, maybe...
for (word, tag) in tagged:
    nltk.help.upenn_tagset( tag )

For the rest of this practical session, you need a few nltk ressources. If not done yet (from [the first practical session](http://localhost:8888/notebooks/03-Tagging/TP1.php#nltk), please do:

In [ ]:
 ## if not downloaded yet (see first practical session)
nltk.download('brown')
nltk.download('universal_tagset')
nltk.download('wordnet')

Then it might also be usefull to already import some symbols:

In [ ]:
from nltk.corpus import brown
from nltk.corpus import wordnet
from nltk.stem.wordnet import WordNetLemmatizer

## 2. Vocabulary size and variability reduction

In this exercise, we will be exploring the interplay between PoS tagging, lemmatization, and vocabulary reduction.

Imagine you have a large corpus and a large number of queries of the form “_Give me all documents in which all of the words x, y and  z appear_”. One way to answer such queries efficiently is to index the documents in the corpus: for instance implement a function called `index()` for which `index(document, word)` returns non-zero if and only if word occurs in document at least once.

One way to implement this kind of index is with a matrix in which each line corresponds to a document and each column corresponds to a word. When the query above is executed, only the lines for which the entries on the x, y and z columns are simultaneously not zero are returned.

The size of this matrix is number of documents × number of distinct words that appear in the corpus. Clearly, the smaller the matrix, the faster the queries are executed. Ideally, we would like to reduce the number of words while not losing the ability to answer queries. One way to do this is to eliminate morphological variability — for instance, map both “_goes_” and “_went_” to the same column “_go_”. This is called "_lemmatization_". Part-of-speech tagging can help lemmatization so as not to confuse similar surface forms with different roles (e.g. not confuse "_can_" as in "_a can of beer_" as opposed to "_I can do it_".).

* To begin, we will need a corpus to work with. Let's make use of the [Brown Corpus](http://en.wikipedia.org/wiki/Brown_Corpus). Load the list of all word-tag tuples from the corpus:

In [ ]:
brown_tagged = brown.tagged_words(tagset="universal")

and see what you got; e.g.

In [ ]:
brown_tagged[0:10]

Note that we are using the “[universal](http://www.nltk.org/book/ch05.html#tab-universal-tagset)” tagset because the original Brown tagset is somewhat unwieldy.

* Next, we want to see how many distinct words appear in the corpus. Variations of the same word should be counted separately, e.g. "_go_" and "_went_" shall be counted separately here, but case should be ignored. One way to do this is to use the NLTK-supplied [`FreqDist` class](http://www.nltk.org/_modules/nltk/probability.html):

In [ ]:
fdist_before = nltk.FreqDist(word.lower() for (word, tag) in brown_tagged)

This builds a frequency distribution from the words in the corpus (have a look!).
To see how many distinct words are in the distribution, simply take the length: 

In [ ]:
len(fdist_before) ## should be 49815

* Now run the [WordNet Lemmatizer](https://www.nltk.org/_modules/nltk/stem/wordnet.html) on the tagged corpus.
  Note that this lemmatizer needs information about the part of speech of a token in order to correctly lemmatize it.
  One way of lemmatizing the entire corpus is to iterate through the list of (token, tag) pairs — `brown_tagged` variable above —, lemmatize each token and add the lemma to a separate list.
  For the part-of-speech format needed for lemmatization, you might find the following function useful (do not forget to `import wordnet from nltk.corpus`):

In [ ]:
# converts a pos tag from the 'universal' format to wordnet format 
def get_wordnet_pos(universal_tag):
    if universal_tag == 'VERB':
        return wordnet.VERB
    elif universal_tag == 'ADJ':
        return wordnet.ADJ
    elif universal_tag == 'ADV':
        return wordnet.ADV
    else:
        return wordnet.NOUN

* Count the distinct lemmatized words, for example using FreqDist. Do not forget to convert everything to lowercase.
  What percentage reduction did you obtain?

[**to do:** write Python code to perform lemmatization of the Brown corpus and answer the above question.]

In [ ]:
lem = WordNetLemmatizer()
 
# build a frequency distribution from the lowercase form of the lemmas
fdist_after = nltk.FreqDist(lem.lemmatize(word.lower(), get_wordnet_pos(tag)) for (word, tag) in brown_tagged)
 
# how many words exist after lemmatization (we obtained ~21% reduction)
print(len(fdist_after)) ## should be 39358
 
print(100 * (len(fdist_before) - len(fdist_after)) / len(fdist_before)) ## should be 21%

## 3. N-gram tagging
### 3.1 Unigram tagger

In this exercise, we will see how adding context can improve the performance of automatic part-of-speech tagging. From the NLTK point of view, everything you need to know can be found in [section 5 of chapter 5 of the book](http://www.nltk.org/book/ch05.html#n-gram-tagging).

We will begin with a simple unigram tagger and build it up to a slightly more complex tagger. Unigram taggers are based on a simple statistical algorithm: for each token, assign the tag that is most likely for that particular token.

* The first thing we want to do is to consider a corpus to work with and separate it into a training set and a test set. We recommend the Brown corpus again, but this time it is a good idea to restrict it to one or two of its categories, otherwise the experiments would be too slow (we choose news and fiction categories below):


In [100]:
#import the tagged and untagged sentences
brown_tagged_sents = brown.tagged_sents(categories=['news', 'fiction'])
print("Size of corpus: {0} sentences".format(len(brown_tagged_sents)))

# split the sentences into training and test sets
size = int(len(brown_tagged_sents) * 0.9)
train_sents = brown_tagged_sents[:size]
print("Size of training set: {0} sentencces".format(size))

# Tagged test sentences (used as reference)
test_sents = brown_tagged_sents[size:]

# Untagged test sentences
raw_sents = brown.sents(categories=['news', 'fiction'])[size:]

Size of corpus: 8872 sentences
Size of training set: 7984 sentencces


* Train a unigram tagger (on the training set of sentences) and run it on some of the sentences from `raw_sents`. Observe that some words are not assigned a tag.
  Why not?
* What is the accuracy of the tagger on the test set (the [`evaluate()` function](http://www.nltk.org/api/nltk.tag.html#nltk.tag.api.TaggerI.evaluate) may come in handy here)?

[**to do:** write Python code to perform unigram tagger learning on the Brown corpus and answer the above questions.]

In [ ]:
# train the tagger and test it on unseen sentences
unigram_tagger = nltk.UnigramTagger(train_sents)
print("Unigram tagger unseen sentence example:")
print(unigram_tagger.tag(raw_sents[10]))

print("\nPerf. of unigram tagger alone: {0}".format(unigram_tagger.evaluate(test_sents)))

### 3.2 Guesser

An easy way to mitigate the problem of some words not being assigned tags is to have the unigram tagger fall back on a guesser when it does not know what tag to assign to a word.

As its name suggests, a guesser is a PoS Tagger that assigns a tag to any token (be it a correct word or not). The use of a guesser as a fallback can improve the robustness of the PoS tagging system (i.e. the system will associate a PoS tag to every word, even if it was not seen beforehand, or if it is not in the lexicon).
Another scenario in which a guesser could be helpful is in the initialization phase of the Brill tagging algorithm. In addition, the guesser’s standalone performance could be used as a baseline, when evaluating more complex taggers.

For instance, perhaps the simplest guesser is the one that just assigns the same tag to every word it encounters. This is called a [“_default tagger_”](http://www.nltk.org/api/nltk.tag.html#nltk.tag.sequential.DefaultTagger) in NLTK.

* Instantiate a default tagger and use it to tag the test sentences (`test_sents`) defined in the previous section.
  Do not combine it yet with other taggers at this point.
  Report the accuracy. (Feel free to look at the examples in [sections 4](http://www.nltk.org/book/ch05.html#automatic-tagging)  [and 5](http://www.nltk.org/book/ch05.html#n-gram-tagging)  from chapter 5 of the NLTK book.)

In [ ]:
t0 = nltk.DefaultTagger('NN')
print("Perf. default tagger alone: {0}".format(t0.evaluate(test_sents)))

* Then use the default tagger as a fallback for a Unigram tagger. Report the change in accuracy.

In [ ]:
t1 = nltk.UnigramTagger(train_sents, backoff=t0)
print("Perf. unigram tagger, fallback on default tagger: {0}".format(t1.evaluate(test_sents)))

* How would you build a better guesser, while still not relying on any further training data?

**Answer:** We could make use of morphology, e.g. using regular expressions to take some decision on the tag to guess, as done now.

One step further than the default tagger could be to assign tags to tokens on the basis of matching patterns. In other words, the form of the token is considered when guessing its part-of-speech. For example, it might guess that any word ending in “ed” is the past participle of a verb, and any word ending with “'s” is a possessive noun. This is called a [“_regular expression tagger_”](http://www.nltk.org/api/nltk.tag.html#nltk.tag.sequential.RegexpTagger) and is already available in NLTK.

* Instantiate a regular expression tagger and use it to tag the test sentences. You could use the same rules provided in the link above, or define some other rules of your own.
  Do not combine it yet with other taggers at this point.
  Report the accuracy.
* Now use the regular expression tagger as a fallback for a Unigram tagger.
  Report the change in accuracy.

In [ ]:
regexp_tagger = nltk.RegexpTagger(
     [(r'^-?[0-9]+(.[0-9]+)?$', 'CD'),   # cardinal numbers
      (r'(The|the|A|a|An|an)$', 'AT'),   # articles
      (r'.*able$', 'JJ'),                # adjectives
      (r'.*ness$', 'NN'),                # nouns formed from adjectives
      (r'.*ly$', 'RB'),                  # adverbs
      (r'.*s$', 'NNS'),                  # plural nouns
      (r'.*ing$', 'VBG'),                # gerunds
      (r'.*ed$', 'VBD'),                 # past tense verbs
      (r'.*', 'NN')                      # nouns (default)
     ])
print("Perf. RegExp tagger alone: {0}".format(regexp_tagger.evaluate(test_sents)))

# evaluate a unigram tagger with a backoff to a regexp tagger
uni_regexp = nltk.UnigramTagger(train_sents, backoff=regexp_tagger)
print("Perf. unigram tagger, fallback on regexp_tagger: {0}".format(uni_regexp.evaluate(test_sents)))

### 3.3 Bigrams and More

When performing PoS tagging using a unigram-based tagger, the larger context in which the words appear is ignored. Unigram taggers consider only one item of context, in isolation. This poses a significant limitation on any tagger in this class: the most that it can do is to assign the a priori most likely tag to a given token. In a sense, the tag of each word is decided before it is placed in a sentence.  Therefore, the word “_object_” would be assigned the same part of speech every time, regardless if it appears as “_the object_” (should be a noun) or as “_to object_” (should be a verb).  

An n-gram tagger picks the tag that is most likely in the given context of size n. An instance of an n-gram tagger is the bigram tagger, which considers groups of two tokens when deciding on the parts-of-speech. For example, we would expect a bigram tagger to find the correct tags for the two occurrences of the word “_objec_t” in the following sentence (as opposed to a unigram tagger):

“_I did not object to the object_”

Indeed, a NLTK unigram tagger trained on the Brown corpus yields: 

In [101]:
sentence = "I did not object to the object"
tokens =  nltk.word_tokenize(sentence)

print(unigram_tagger.tag(tokens))
print(t1.tag(tokens))
print(uni_regexp.tag(tokens))

[('I', 'PPSS'), ('did', 'DOD'), ('not', '*'), ('object', 'VB'), ('to', 'TO'), ('the', 'AT'), ('object', 'VB')]
[('I', 'PPSS'), ('did', 'DOD'), ('not', '*'), ('object', 'VB'), ('to', 'TO'), ('the', 'AT'), ('object', 'VB')]
[('I', 'PPSS'), ('did', 'DOD'), ('not', '*'), ('object', 'VB'), ('to', 'TO'), ('the', 'AT'), ('object', 'VB')]


No way to have a different tag for the two "_object_" words.

* Train a bigram tagger alone on the training set (`train_sents`) and check it in the above sentence (weird result expected, see below).
* Then, find its accuracy on the test set (`test_sents`).

In [ ]:
bigram = nltk.BigramTagger(train_sents)

print(bigram.tag(tokens))

print("\nPerf. bigram tagger alone: {0}".format(bigram.evaluate(test_sents)))

You have probably found a surprising result on our sample sentence and very low accuracy for the bigram tagger, when run alone. If you run the trained bigram tagger on a sentence it has not seen during training (e.g. bigram.tag(raw_sents[1]), you will notice that a majority of words will be tagged as “None”. When the bigram tagger encounters a new word, it is unable to assign a tag. It will also be unable to tag the following word — even if it was seen during training — because it was never seen preceded by a “None” tag. From this point on, all the words up to the end of the sentence will be tagged as “None” for the same reason. This explains the low accuracy of the bigram tagger when run alone.

The problem that arises when using n-grams is that once we pass to larger values of n, more data is needed for training, as the specificity of the context increases. In other words, there are many more possible bigrams than unigrams and thus the word/tag combinations that we need to consider when tagging are less likely to have appeared in the training set, as n increases. This is known as the sparse data problem, and is quite pervasive in NLP.

This means that, when using n-gram taggers, we encounter the following tradeoff: having n large means more accurate predictions, at the cost of having to provide more training data; having a smaller n, leads to not so accurate predictions, but less necessary training data.

In order to overcome this limitation, we can train a bigram tagger that falls back on our “unigram+default” or “unigram+regexp” taggers defined in the previous section.

* Try this out now (an example for a bigram tagger with fallback on “unigram+default” can be found in the book, [section 5.5.4 “_Combining Taggers_”](http://www.nltk.org/book/ch05.html#combining-taggers)). Does this improve performance?
* What about adding a trigram tagger that falls back on the “bigram+unigram+default” or on “bigram+unigram+regexp”?  

In [ ]:
# evaluate a bigram tagger with a backoff to the unigram+dummy tagger
t2 = nltk.BigramTagger(train_sents, backoff=t1)
print("Perf. bigram tagger,  fallback on        unigram+default tagger: {0}".format(t2.evaluate(test_sents)))
 
# evaluate a trigram tagger with a backoff to the bigram+unigram+dummy tagger
t3 = nltk.TrigramTagger(train_sents, backoff=t2)
print("Perf. trigram tagger, fallback on bigram+unigram+default tagger: {0}".format(t3.evaluate(test_sents)))
 
print("")
# evaluate a bigram tagger with a backoff to the unigram+regexp tagger
t2_regexp = nltk.BigramTagger(train_sents, backoff=uni_regexp)
print("Perf. bigram tagger,  fallback on        unigram+regexp tagger: {0}".format(t2_regexp.evaluate(test_sents)))
 
# evaluate a trigram tagger with a backoff to the bigram+unigram+regexp tagger
t3_regexp = nltk.TrigramTagger(train_sents, backoff=t2_regexp)
print("Perf. trigram tagger, fallback on bigram+unigram+regexp tagger: {0}".format(t3_regexp.evaluate(test_sents)))

## and on our sample sentence:
print(t2.tag(tokens))

## 4. Viterbi Algorithm

Code the Viterbi algorithm (in your most favorite programming language) and apply it to the toy example provided on slide 8 of the lecture on HMM: two symbols (H,T), two states (the two coins),  
A = [[ 0.4 0.6 ]; [0.9 0.1 ]],  
B = [[ 0.49 0.51 ]; [ 0.85 0.15 ]],  
I = [ 0.5 0.5 ].

What is (one of) the most probable sequence of states corresponding to the observation `HTTHTTHHTTHTTTHHTHHTTHTTTTHTHHTHTHHTTTH`?

[Here (link)](https://coling.epfl.ch/TP/corr/viterbi.pdf) are some slides illustrating the Viterbi algorithm at work on a concreate simple example.

Regarding our question, the solution should be one of 32 _equaly probable_ solutions:
```
211211 12 112111 12 1 12 112111121 12 121 12 1112
211211 12 112111 12 1 12 112111121 12 121 21 1112
211211 12 112111 12 1 12 112111121 21 121 12 1112
211211 12 112111 12 1 12 112111121 21 121 21 1112
211211 12 112111 12 1 21 112111121 12 121 12 1112
211211 12 112111 12 1 21 112111121 12 121 21 1112
211211 12 112111 12 1 21 112111121 21 121 12 1112
211211 12 112111 12 1 21 112111121 21 121 21 1112
                                             
211211 12 112111 21 1 12 112111121 12 121 12 1112
211211 12 112111 21 1 12 112111121 12 121 21 1112
211211 12 112111 21 1 12 112111121 21 121 12 1112
211211 12 112111 21 1 12 112111121 21 121 21 1112
211211 12 112111 21 1 21 112111121 12 121 12 1112
211211 12 112111 21 1 21 112111121 12 121 21 1112
211211 12 112111 21 1 21 112111121 21 121 12 1112
211211 12 112111 21 1 21 112111121 21 121 21 1112
                                             
211211 21 112111 12 1 12 112111121 12 121 12 1112
211211 21 112111 12 1 12 112111121 12 121 21 1112
211211 21 112111 12 1 12 112111121 21 121 12 1112
211211 21 112111 12 1 12 112111121 21 121 21 1112
211211 21 112111 12 1 21 112111121 12 121 12 1112
211211 21 112111 12 1 21 112111121 12 121 21 1112
211211 21 112111 12 1 21 112111121 21 121 12 1112
211211 21 112111 12 1 21 112111121 21 121 21 1112
                                             
211211 21 112111 21 1 12 112111121 12 121 12 1112
211211 21 112111 21 1 12 112111121 12 121 21 1112
211211 21 112111 21 1 12 112111121 21 121 12 1112
211211 21 112111 21 1 12 112111121 21 121 21 1112
211211 21 112111 21 1 21 112111121 12 121 12 1112
211211 21 112111 21 1 21 112111121 12 121 21 1112
211211 21 112111 21 1 21 112111121 21 121 12 1112
211211 21 112111 21 1 21 112111121 21 121 21 1112
```

Here are [a C++ code](https://coling.epfl.ch/TP/corr/viterbi.cc), [a Perl code](https://coling.epfl.ch/TP/corr/viterbi.pl) and [an Haskel code (from 2014 student Romain Edelman)](https://coling.epfl.ch/TP/corr/Viterbi.hs).